# 通过 RPC 查询数据

单击顶部栏上的`🚀` -&gt; `Binder`在线运行此示例！

## 准备

In [2]:
# 准备阶段: 初始化 w3 对象
from conflux_web3 import Web3

w3 = Web3(Web3.HTTPProvider("https://test.confluxrpc.com"))

# 准备后文需要的常量
transaction_hash = "0x10a721e2654523a7ff682c1c8d3f868a9fdf78a3be1858bbe1f06147128d6d94"
epoch = 98943500

## 通过 RPC 查询数据

我们可以使用 SDK 通过 JSON-RPC 来查询区块链状态、交易数据、区块数据等。完整的 RPC 文档可以在[这里](https://developer.confluxnetwork.org/conflux-doc/docs/json_rpc/#json-rpc-methods)找到。

### 查询区块链状态

In [3]:
# get_status 提供了区块链当前状态的概览
w3.cfx.get_status()

AttributeDict({'bestHash': HexBytes('0xfccec01b7d00bf7c60039ac24e832cda36c3f99847d85bdebea5ad9c23e72af0'),
 'chainId': 1,
 'ethereumSpaceChainId': 71,
 'networkId': 1,
 'epochNumber': 99720385,
 'blockNumber': 127806496,
 'pendingTxNumber': 0,
 'latestCheckpoint': 99640000,
 'latestConfirmed': 99720321,
 'latestState': 99720381,
 'latestFinalized': 99719940})

多个 RPC 方法都可以接受**epoch number 参数**。`epoch number`（纪元）参数可以是：

- 一个整数
- 或者是字符串形式的 epoch tag`latest_mined` , `latest_state` , `latest_confirmed`或`latest_finalized`

Conflux 中的`epoch number`的概念类似于其他区块链中的`block number`的概念，但一个 epoch 包含一个或多个区块。详细信息请参阅 https://developer.confluxnetwork.org/conflux-doc/docs/json_rpc#the-default-epochnumber-parameter。

In [4]:
# latest_mined epoch number
print(f"latest_mined epoch number: {w3.cfx.epoch_number}")
# 通过 tag 获取纪元
print(f'latest_state epoch numebr: {w3.cfx.epoch_number_by_tag("latest_state")}')

latest_mined epoch number: 99720385
latest_state epoch numebr: 99720381


### 查询交易

In [5]:
# 通过交易哈希获取交易
# 交易发出后这个 RPC 立刻可用
w3.cfx.get_transaction_by_hash(transaction_hash)

AttributeDict({'hash': HexBytes('0x10a721e2654523a7ff682c1c8d3f868a9fdf78a3be1858bbe1f06147128d6d94'),
 'nonce': 2,
 'blockHash': HexBytes('0xf0ac5f8d757a7415876e322a245d5eff76349b656b3eadb10d5e09a4a3f7d59d'),
 'transactionIndex': 0,
 'from': 'cfxtest:aan2vyszrgz3sr3tug73ywb8k128y5wa7yhpjm1hn1',
 'to': 'cfxtest:acejjfa80vj06j2jgtz9pngkv423fhkuxj786kjr61',
 'value': 0 Drip,
 'gasPrice': 1000000000 Drip,
 'gas': 41633,
 'contractCreated': None,
 'data': HexBytes('0x5f3cc9f7'),
 'storageLimit': 0,
 'epochHeight': 87083312,
 'chainId': 1,
 'status': 0,
 'v': 1,
 'r': HexBytes('0x17b30481d63a49223092215a2aa503938519df5cac0d20cbff7ebf50569776cd'),
 's': HexBytes('0x5fc96a1e383879cf66c09d106b2df4907c674a263c9a7bba602078c90b0038b1')})

In [6]:
# 通过交易哈希获取交易receipt
# 交易执行后才能获取到 receipt 
w3.cfx.get_transaction_receipt(transaction_hash)

AttributeDict({'transactionHash': HexBytes('0x10a721e2654523a7ff682c1c8d3f868a9fdf78a3be1858bbe1f06147128d6d94'),
 'index': 0,
 'blockHash': HexBytes('0xf0ac5f8d757a7415876e322a245d5eff76349b656b3eadb10d5e09a4a3f7d59d'),
 'epochNumber': 87083316,
 'from': 'cfxtest:aan2vyszrgz3sr3tug73ywb8k128y5wa7yhpjm1hn1',
 'to': 'cfxtest:acejjfa80vj06j2jgtz9pngkv423fhkuxj786kjr61',
 'gasUsed': 35480,
 'gasFee': 35480000000000 Drip,
 'contractCreated': None,
 'logs': [],
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'

### 查询块

在 Conflux 中，每个 epoch 包含 1 个或多个区块，其中一个块称为`pivot`区块。`pivot`区块确定各个 epoch 中都包含哪些区块。

In [7]:
# 查询某个 epoch 中包含的区块
block_hashes = w3.cfx.get_blocks_by_epoch(epoch)
print(f"blocks in epoch {epoch}:\n{block_hashes}")
# block_hashes 中最后一个元素一定为 pivot 区块
print(f"pivot block hash of epoch {epoch}: {block_hashes[-1].hex()}")

blocks in epoch 98943500:
[HexBytes('0xae622377e139608a9202504df4bf20570c4741370d36f668952ae91c57c5da69'), HexBytes('0x104a3549108c7c83a36c65d188e842f6deb9a770d3ca749a222fce628846c04b')]
pivot block hash of epoch 98943500: 0x104a3549108c7c83a36c65d188e842f6deb9a770d3ca749a222fce628846c04b


In [8]:
# 如果某个区块为 pivot 区块, 我们可以通过 epoch 查询到该区块
# 当然，我们也可以通过哈希查询区块
assert w3.cfx.get_block_by_epoch_number(epoch) == w3.cfx.get_block_by_hash(block_hashes[-1])

In [9]:
# get_block_by_epoch_number 也支持通过 epoch tag 查询
w3.cfx.get_block_by_epoch_number("latest_state")

AttributeDict({'hash': HexBytes('0x37ab44387ae132e3eea6c666bfb5cd58af50d0b172ab21b0be5ff85073bc5592'),
 'parentHash': HexBytes('0x9130a237d15b696d48daf1decf6f2908a98a57c9d2b543d1cce7f55c296cf7fd'),
 'height': 99720383,
 'miner': 'cfxtest:aang4d91rejdbpgmgtmspdyefxkubj2bbywrwm9j3z',
 'deferredStateRoot': HexBytes('0x48edc4d8dbe77ac589cf30087ad3165ee533101882698812f9f4bcff0ff66252'),
 'deferredReceiptsRoot': HexBytes('0x09f8709ea9f344a810811a373b30861568f5686e649d6177fd92ea2db7477508'),
 'deferredLogsBloomHash': HexBytes('0xd397b3b043d87fcd6fad1291ff0bfd16401c274896d8c63a923727f077b8e0b5'),
 'blame': 0,
 'transactionsRoot': HexBytes('0xc5d2460186f7233c927e7db2dcc703c0e500b653ca82273b7bfad8045d85a470'),
 'epochNumber': 99720383,
 'blockNumber': 127806493,
 'gasLimit': 30000000,
 'gasUsed': 0,
 'timestamp': 1668507784,
 'difficulty': 58688104,
 'powQuality': HexBytes('0x12b15fcb'),
 'refereeHashes': [HexBytes('0xb18df88debc7ab3fb8a2dc105e927d9f2bc6575b5a38c5e84b5c84c7c41b5525')],
 'adaptiv

In [10]:
# 通过 block number 查询也是可行的
w3.cfx.get_block_by_block_number(110979562)

AttributeDict({'hash': HexBytes('0xf0ac5f8d757a7415876e322a245d5eff76349b656b3eadb10d5e09a4a3f7d59d'),
 'parentHash': HexBytes('0xeaf2f6227c068ae27cb847657af7eb7e3cfd6696f5a795d2f38159074d20a8c9'),
 'height': 87083316,
 'miner': 'cfxtest:aaskvgxcfej371g4ecepx9an78ngrke5ay9f8jtbgg',
 'deferredStateRoot': HexBytes('0xa26347f02f312db0fba655c29b9fa853e8e6242449a67added74c62679122f62'),
 'deferredReceiptsRoot': HexBytes('0x09f8709ea9f344a810811a373b30861568f5686e649d6177fd92ea2db7477508'),
 'deferredLogsBloomHash': HexBytes('0xd397b3b043d87fcd6fad1291ff0bfd16401c274896d8c63a923727f077b8e0b5'),
 'blame': 0,
 'transactionsRoot': HexBytes('0xdc62bb0e7115a9727688811f4e090efdfd203a516953ff7829e21dea29e1f166'),
 'epochNumber': 87083316,
 'blockNumber': 110979562,
 'gasLimit': 30000000,
 'gasUsed': 35480,
 'timestamp': 1660093852,
 'difficulty': 54757256,
 'powQuality': HexBytes('0x0365f39d'),
 'refereeHashes': [],
 'adaptive': False,
 'nonce': HexBytes('0xe72421314ac723e6'),
 'size': 113,
 'custo

### 其他 RPC

除此之外， `conflux-web3`支持 cfx 命名空间下的所有 rpc 方法。[JSON-RPC-methods](https://developer.confluxnetwork.org/conflux-doc/docs/json_rpc#json-rpc-methods)中提供了更多信息。

这里提供了部分例子：

In [11]:
w3.cfx.gas_price

1000000000 Drip

In [12]:
len(w3.cfx.get_code("cfxtest:acejjfa80vj06j2jgtz9pngkv423fhkuxj786kjr61"))

5964